# KD Tree for Bottleneck TSP

In [1]:
%load_ext autoreload
%autoreload 2

## Get Spotify Tracks

In [2]:
import tekore as tk
from smoothify.features import SpotifyFeatureConstructor
import numpy as np

### Obtaining an API Access Token

You'll need an access token to call the Spotify API.

Here's how to get a temporary token to play around with (expires in ~30 mins):

- Visit https://developer.spotify.com/console/get-playlist-tracks/
- Click 'Get Token'
- Select the following permissions:
    - `user-library-read`
    - `playlist-read-private`
    - `playlist-read-collaborative`
    - `playlist-modify-public`
    - `playlist-modify-private`
- Click 'Request Token'
- Copy the text in the 'OAuth Token' field

Paste the access token below

In [4]:
ACCESS_TOKEN = "BQAmixZq0_OCgzh2PAdH73EJqgaNkxCwJ5sUi4yDr3iQNDvhvooL7hArn2Hf1xSpeWtO_ALJg1zYDUDiRvTo5J5rP8sEagw3txioGb1HV_Y6xmxAWtXKibhoBRRIk4PZ26DQgvSUGrPh5nYxnj6tqtX_s0jHAE1XhyFpbLm-0Hu95fvrtib0gnlpxxgF6V9_v6PXeVowalQvG6XOA2VubYmRJ6ruRagvuVhvoc1160HmzETkY8c"
spotify = tk.Spotify(ACCESS_TOKEN, asynchronous=True, max_limits_on=True, chunked_on=True)
current_user = await spotify.current_user()

### Fetch tracks from current user's Liked Songs

In [5]:
saved_tracks_page = await spotify.saved_tracks()
track_list = [track.track async for track in spotify.all_items(saved_tracks_page)]
playlist_name = f"{current_user.display_name}'s Library"
is_playlist_public = False
is_playlist_collaborative = False
print(f"Fetched {len(track_list)} tracks")

Fetched 532 tracks


### Fetch tracks from a playlist

To read the tracks in a playlist, you'll need the playlist's ID.

You can find a playlist's ID in its URL, which looks like `https://open.spotify.com/playlist/{playlist_id}`

#### Getting a Playlist's URL

- Navigate to your playlist
- Under the playlist's title, click the triple-dot icon and select to `Share -> Copy link to playlist`


In [ ]:
PLAYLIST_ID = "78RmalWaNX12cJSmfVHlXL"
playlist = await spotify.playlist(PLAYLIST_ID)
track_list = [track.track async for track in spotify.all_items(playlist.tracks)]
playlist_name = playlist.name
is_playlist_public = playlist.public
is_playlist_collaborative = playlist.collaborative
print(f"Fetched {len(track_list)} tracks")

## Construct audio features

### Query Spotify API for audio features

In [6]:
# Extract track IDs so we can fetch additional info
track_id_list = [track.id for track in track_list]
# Get track audio features
track_features_list = await spotify.tracks_audio_features(track_id_list)

### Merge & normalize features

In [8]:
feature_constructor = SpotifyFeatureConstructor(audio_features_list=track_features_list)
features_df = feature_constructor.construct_features()
features_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.146348,-0.215775,0.465682,2.827531,-0.314236,-0.783334,-0.258798,0.492498,1.291638
1,1.527861,-0.558760,-0.351044,0.124693,-0.678228,-0.737701,-0.702651,1.075416,-0.022270
2,-0.270841,1.333190,0.874360,0.333189,-0.564577,-0.783334,-0.356887,1.306986,0.266052
3,0.837104,0.243383,0.154533,-0.756469,-0.296122,-0.783334,1.911424,0.596305,0.182320
4,0.440432,1.189358,0.869326,1.549162,-0.644927,-0.783307,1.065405,0.404660,0.102215


## Compute the best path through the points

In [8]:
from smoothify.optim.bottleneck_tsp import KDTreeBottleneckTSP

In [11]:
points = np.array(features_df)

In [22]:
optimizer = KDTreeBottleneckTSP(points=points)
results = optimizer.get_best_path()
best_path = results.best_path
max_dist = results.min_max_dist
print(f"Max edge length: {max_dist}")

  0%|                                                                                                                                                                                                                                 | 0/531 [00:00<?, ?it/s]

[ProgressParallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 531/531 [00:22<00:00, 23.21it/s]

[ProgressParallel(n_jobs=-1)]: Done 531 out of 531 | elapsed:   22.9s finished
Max edge length: 4.7158256001683645


## Create a smoothified playlist

**NOTE: Running this cell will add a playlist to your Spotify account**

In [23]:
# Create new playlist
NEW_PLAYLIST_NAME = f"{playlist_name} (but smoother)"
new_playlist = await spotify.playlist_create(current_user.id, NEW_PLAYLIST_NAME, public=is_playlist_public)
await spotify.playlist_change_details(new_playlist.id, collaborative=is_playlist_collaborative)

# Add tracks to new playlist
smoothified_track_uris = [track_list[node_idx].uri for node_idx in best_path]
await spotify.playlist_add(new_playlist.id, smoothified_track_uris)

'NywyMmYyY2QxNTY1MTAzZjc1NjM3ZTM4NTgyMWY1MDNiZGNjYmNiNDkx'

## Visualize paths

In [12]:
from sklearn.manifold import TSNE

In [13]:
reducer = TSNE(learning_rate="auto", init="pca")
points_2d = reducer.fit_transform(points)

/home/krishna/.pyenv/versions/3.9.9/envs/smoothify/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [14]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool


def make_plot(*, x, y, path):
    p = figure(tools="hover")
    # Data source
    data = dict(x=x, y=y)
    # Hover tooltips
    hover = p.select({"type": HoverTool})
    hover.tooltips = {k: f"@{k}" for k in data}
    # Draw points
    p.scatter(source=ColumnDataSource(data=data), x="x", y="y", radius=0.5, fill_alpha=0.5, line_color=None)
    # Draw path
    p.line(x[path], y[path], alpha=0.33)
    return p


initial_order = make_plot(x=points_2d[:, 0], y=points_2d[:, 1], path=list(range(len(points_2d))))
best_order = make_plot(x=points_2d[:, 0], y=points_2d[:, 1], path=best_path)

output_notebook()
show(initial_order)
show(best_order)

Loading BokehJS ...